In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import keras
import keras_nlp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

c:\Users\user\Desktop\BERTmodel\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df = pd.read_parquet("hf://datasets/dair-ai/emotion/unsplit/train-00000-of-00001.parquet")
df

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,text,label
0,i feel awful about it too because it s my job ...,0
1,im alone i feel awful,0
2,ive probably mentioned this before but i reall...,1
3,i was feeling a little low few days back,0
4,i beleive that i am much more sensitive to oth...,2
...,...,...
416804,that was what i felt when i was finally accept...,1
416805,i take every day as it comes i m just focussin...,4
416806,i just suddenly feel that everything was fake,0
416807,im feeling more eager than ever to claw back w...,1


In [ ]:
grouped = df.groupby("label")
min_count = grouped.size().min()

balanced_df = grouped.apply(lambda x: x.sample(min_count, random_state=42))

# 4) Reset the index (because groupby + apply modifies it)
balanced_df = balanced_df.reset_index(drop=True)
X = balanced_df['text']
Y = balanced_df['label']
X = [str(item) for item in X]
le = LabelEncoder()
balanced_df["label"] = le.fit_transform(balanced_df["label"])
print(le.classes_)  # Ensure you have 6 unique classes
print(balanced_df["label"].unique())

[0 1 2 3 4 5]
[0 1 2 3 4 5]


<ipython-input-4-be5999762ba1>:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df = grouped.apply(lambda x: x.sample(min_count, random_state=42))


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify = Y)

In [ ]:
X_train = np.array(X_train, dtype=object)
X_test = np.array(X_test, dtype=object)
y_train = np.array(y_train, dtype=np.int32)
y_test = np.array(y_test, dtype=np.int32)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(71865,) (71865,)
(17967,) (17967,)


In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text_input")

preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_base_en_uncased",
    sequence_length=128,  # Adjust if you need longer sequences
    # Typically not trainable, but you can set it if you want
    trainable=False
)

encoder_inputs = preprocessor(text_input)

encoder = keras_nlp.models.BertBackbone.from_preset(
    "bert_base_en_uncased",
    load_weights=True,
    trainable=True
)

# Extract the BERT outputs
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]

100%|██████████| 510/510 [00:00<00:00, 1.13MB/s]


100%|██████████| 547/547 [00:00<00:00, 381kB/s]


100%|██████████| 226k/226k [00:00<00:00, 1.04MB/s]


100%|██████████| 418M/418M [00:09<00:00, 48.5MB/s]


In [ ]:
dropout = tf.keras.layers.Dropout(0.1)(pooled_output)
classifier = tf.keras.layers.Dense(6, activation="softmax")(dropout)

# 8. Construct the model
model = tf.keras.Model(inputs=text_input, outputs=classifier)
model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
    jit_compile=False
    )

In [ ]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=2,
    batch_size=64
)


# 8. Evaluate
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/2
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 2091s 2s/step - accuracy: 0.6520 - loss: 0.8574 - val_accuracy: 0.9487 - val_loss: 0.1410
Epoch 2/2
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 2056s 2s/step - accuracy: 0.9501 - loss: 0.1306 - val_accuracy: 0.9521 - val_loss: 0.1223
Test Loss: 0.1223
Test Accuracy: 0.9521
